# Decoding with Belief Propagation - Working to take some for loops out of the game

We want to iterate the following the following equations:

$$ \hat{m}_{\mu j} = \tanh ( \beta(\rho) J_\mu ) \prod_{ l \in {\cal L} (\mu) \backslash j } m_{\mu l}  \; \; ,  $$ 

$$ m_{\mu j} = \tanh \left(  \sum_{ \nu \in {\cal M} (j) \backslash \mu } \ \tanh^{-1} \left(  \hat{m}_{\nu j}  \right)    + \beta(\rho_\xi)  \right)   \; \; ,  $$  

which are Eqs.(96) from the paper [Low-density parity-check codes—A statistical physics perspective](https://www.sciencedirect.com/science/article/pii/S1076567002800180), by R. Vicente, D. Saad and Y. Kabashima.

The function $ \beta(x) $ is the Nishimori temperature,

$$ \beta(x) = \frac{1}{2} \log \left(  \frac{1- \rho}{\rho}  \right) \; \; .  $$

The quantity $\rho$ is the flip probablility of the noisy channel (BSC),

$$ P ( J | J^{(0)} ) = (1 - \rho) \delta_{J, J^{(0)} } + \rho \delta_{J, -J^{(0)} }   \; \; , $$

whereas the prior distribution for each meassage bit is assumed to be

$$ P ( S_j ) = (1 - \rho_\xi) \delta_{+1, S_j }  +  \rho_\xi \delta_{-1, S_j }  \; \; .  $$

The object $ {\cal L} (\mu)$ represents the set of $K$ non-zero elements on the row $\mu$ of the code generator matrix ${\cal G}$ (the one which adds redundancy), 

$$  {\cal L} (\mu) = \langle i_1, i_2, ..., i_K \rangle  \; \; .  $$

The are $C$ non-zero elements per column on the matrix ${\cal G}$:

$$ \sum_{\mu : j \in {\cal L}(\mu)} i_j = C \; \; ; \; \; \forall j = 1, ..., K \; \; . $$

The object $ {\cal M} (j)$ represents the set of all index sets that contain $j$.

After convergence of the iterative procedure, we can calculate the pseudo-posterior:

$$ m_{j} = \tanh \left(  \sum_{ \nu \in {\cal M} (j) } \tanh^{-1} \left(  \hat{m}_{\nu j}  \right)    + \beta(\rho_\xi)  \right)   \; \; ,  $$
from which the Bayes optimal estimate is obtained
$$ \hat{\xi}_j  = sign( m_j ) $$ 

In [17]:
import torch
import numpy as np
import matplotlib.pyplot as plt

Defining the variables of the problem:

In [301]:
# Message lengh
N = 100

# Codeword lengh
M = 200

# Non-zero elements per row of the generation matrix
K = 4

# Number of messages
n = 10

# Noisy channel
p = 0.1
beta = 0.5*np.log( (1 - p) / p)

# Message prior
p_prior = 0.4
beta_prior = 0.5*np.log( (1 - p_prior) / p_prior)

## Generating messages

Each message is a $N$ dimensional vector. Generate a set of $n$ messages.

In [302]:
random = torch.rand([n, N])

#### -1 with probability p_prior
#### +1 with probability 1 - p_prior

message = 2* (random > p_prior).float() - 1

In [303]:
print(message)
message.shape

tensor([[-1.,  1.,  1., -1., -1.,  1., -1.,  1., -1.,  1.,  1., -1.,  1.,  1.,
         -1., -1., -1., -1.,  1., -1., -1., -1.,  1., -1.,  1., -1.,  1., -1.,
          1.,  1.,  1., -1.,  1.,  1., -1.,  1.,  1., -1.,  1., -1.,  1., -1.,
          1., -1., -1., -1., -1.,  1., -1.,  1.,  1.,  1., -1.,  1.,  1., -1.,
          1.,  1., -1.,  1., -1., -1.,  1., -1.,  1.,  1.,  1., -1.,  1.,  1.,
         -1.,  1.,  1.,  1., -1., -1.,  1., -1., -1.,  1., -1.,  1., -1.,  1.,
          1., -1.,  1.,  1.,  1.,  1., -1., -1., -1.,  1., -1., -1.,  1.,  1.,
         -1., -1.],
        [ 1.,  1., -1.,  1.,  1.,  1.,  1., -1.,  1., -1., -1.,  1.,  1.,  1.,
          1.,  1., -1., -1., -1., -1.,  1.,  1., -1.,  1.,  1., -1., -1., -1.,
         -1.,  1., -1., -1., -1.,  1.,  1.,  1.,  1., -1.,  1., -1., -1.,  1.,
          1., -1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1.,  1.,  1., -1.,
         -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1., -1.,  1., -1.,
         -1.,  1.,  1., -1.,  1.

torch.Size([10, 100])

## Encoding

Each message is encoded to a high dimensional vector ${\bf J}^{(0)} \in \{ \pm 1  \}^M$ defined as 

$$   J^{(0)}_{\langle i_1, i_2, ...., i_K \rangle} = \xi_1 \xi_ 2 ... \xi_K  \; \; ,$$

where $M$ sets of $K \in [ 1, ..., N]$ indexes are randomly chosen.

In [304]:
encoding = torch.randint(0, N, [M, K])

In [305]:
encoding.shape

torch.Size([200, 4])

From `encoding`, we construct the encoded message ${\bf J}^{(0)}$.

In [306]:
# Initializing
J0 = torch.take(message[0], encoding).prod(dim=1)
J0 = J0.unsqueeze(0)

# Loop over all messages
for j in range(1, message.shape[0]):
    
    J0_ = torch.take(message[j], encoding).prod(dim=1)
    J0_ = J0_.unsqueeze(0)
    
    J0 = torch.cat((J0, J0_), dim= 0)

In [307]:
print(J0)
J0.shape

tensor([[-1.,  1., -1.,  ...,  1.,  1., -1.],
        [ 1., -1., -1.,  ..., -1.,  1.,  1.],
        [ 1.,  1.,  1.,  ...,  1., -1., -1.],
        ...,
        [-1.,  1.,  1.,  ..., -1., -1.,  1.],
        [-1.,  1., -1.,  ..., -1.,  1., -1.],
        [-1., -1.,  1.,  ...,  1., -1.,  1.]])


torch.Size([10, 200])

## Corrupted version of the encoded set of messages

In [308]:
random = torch.rand(J0.shape)

## Flip tensor: -1 with probability p
flip = 2*(random > p).float() - 1

## J corrupted version: element wise multiplication
J = torch.mul(J0, flip)

In [309]:
print(J)
J.shape

tensor([[-1.,  1., -1.,  ...,  1., -1., -1.],
        [ 1., -1., -1.,  ..., -1.,  1.,  1.],
        [ 1.,  1.,  1.,  ...,  1., -1., -1.],
        ...,
        [ 1.,  1.,  1.,  ...,  1., -1.,  1.],
        [-1.,  1., -1.,  ..., -1.,  1., -1.],
        [-1., -1.,  1.,  ...,  1., -1.,  1.]])


torch.Size([10, 200])

## Iterating BP equations for one message

Let us focus in one received message to iterate the belief propagation equations.

$$ \hat{m}_{\mu j} = \tanh ( \beta(\rho) J_\mu ) \prod_{ l \in {\cal L} (\mu) \backslash j } m_{\mu l}  \; \; ,  $$ 

$$ m_{\mu j} = \tanh \left(  \sum_{ \nu \in {\cal M} (j) \backslash \mu} \ \tanh^{-1} \left(  \hat{m}_{\nu j}  \right) + \beta(\rho_\xi)    \right)   \; \; ,  $$  

with $j = 1, ..., N$ and $\mu = 1, ..., M$.

We cal this message `J_`. We will worry later about a loop over all the received messages.

In [310]:
J_ = J[0]
print(J_)
print(J_.shape)

tensor([-1.,  1., -1.,  1., -1.,  1.,  1., -1.,  1.,  1.,  1., -1., -1., -1.,
         1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1., -1., -1., -1.,
        -1., -1.,  1.,  1., -1., -1., -1., -1., -1., -1., -1.,  1., -1.,  1.,
         1.,  1., -1., -1., -1., -1., -1.,  1., -1., -1., -1., -1., -1., -1.,
        -1.,  1., -1., -1.,  1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        -1., -1., -1.,  1.,  1.,  1.,  1.,  1., -1., -1.,  1.,  1., -1., -1.,
        -1.,  1.,  1.,  1.,  1., -1.,  1., -1., -1.,  1., -1.,  1.,  1., -1.,
         1.,  1.,  1.,  1., -1.,  1.,  1.,  1., -1.,  1., -1.,  1.,  1., -1.,
        -1., -1., -1., -1., -1.,  1., -1.,  1.,  1.,  1., -1., -1.,  1., -1.,
        -1., -1., -1.,  1., -1.,  1., -1., -1., -1., -1.,  1.,  1., -1., -1.,
         1., -1.,  1.,  1., -1.,  1.,  1., -1.,  1.,  1., -1.,  1.,  1., -1.,
         1., -1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1., -1.,  1., -1.,
         1., -1.,  1., -1.,  1.,  1.,  1.,  1., -1., -1., -1., -

The beliefs are $m_{\mu l}$ initialized in order to reflect the prior (from the paper [Low-density parity-check codes—A statistical physics perspective](https://www.sciencedirect.com/science/article/pii/S1076567002800180), by R. Vicente, D. Saad and Y. Kabashima).

In [311]:
#m = torch.ones([M, N])*np.tanh(beta_prior)
m = torch.rand(M, N)

In [312]:
m

tensor([[0.9177, 0.5365, 0.2369,  ..., 0.2376, 0.9661, 0.7244],
        [0.4617, 0.0990, 0.7992,  ..., 0.5979, 0.5452, 0.7027],
        [0.1043, 0.9368, 0.5410,  ..., 0.7527, 0.7192, 0.0162],
        ...,
        [0.1266, 0.1093, 0.5698,  ..., 0.3266, 0.2445, 0.6563],
        [0.3370, 0.6885, 0.9847,  ..., 0.7704, 0.9941, 0.5589],
        [0.3569, 0.7944, 0.2125,  ..., 0.4987, 0.7238, 0.0904]])

Initialize an empty tensor to represent $\hat{m}_{\mu l}$.

In [313]:
#m_hat = torch.ones([M, N])*0.0
m_hat = torch.zeros(M, N)

In [314]:
m_hat

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

We want to calculate $\hat{m}_{\mu j}$.

$$ \hat{m}_{\mu j} = \tanh ( \beta(\rho) J_\mu ) \prod_{ l \in {\cal L} (\mu) \backslash j } m_{\mu l}  \; \; ,  $$ 

This first implementation has two `for` loops. This is potentially harmful if one cares about efficienty. We obviously do, but since we are just beginning, lets go on like this.

In [315]:
for mu in range(M):
    for j in range(N):
          
        # Keep only L(mu) which a are different of j
        index_no_j = torch.nonzero(encoding[mu] != j).squeeze()
        L_no_j = encoding[mu][index_no_j]
        
        # Message update        
        m_hat[mu, j] = torch.tanh( beta* J_[mu])*torch.take(m[mu], L_no_j).prod(dim=0)

In [316]:
m_hat

tensor([[-0.0428, -0.0428, -0.0428,  ..., -0.0428, -0.0428, -0.0428],
        [ 0.0951,  0.0951,  0.0951,  ...,  0.0951,  0.0951,  0.0951],
        [-0.0437, -0.0437, -0.0437,  ..., -0.0437, -0.0437, -0.0437],
        ...,
        [ 0.0154,  0.0154,  0.0154,  ...,  0.0154,  0.0154,  0.0154],
        [-0.1915, -0.1915, -0.1915,  ..., -0.1915, -0.1915, -0.1915],
        [-0.1849, -0.1849, -0.1849,  ..., -0.1849, -0.1849, -0.1849]])

The next step is to implement:

$$ m_{\mu j} = \tanh \left(  \sum_{ \nu \in {\cal M} (j) \backslash \mu} \ \tanh^{-1} \left(  \hat{m}_{\nu j}  \right) + \beta(\rho_\xi)     \right)  \; \; ,  $$ 

In [317]:
for j in range(N):
    
    for mu in range(M):
        
        M_set = torch.where(encoding == j)[0]
        
        M_set_no_mu = M_set[torch.nonzero(M_set != mu).squeeze()]
        
        m[mu, j] = torch.tanh(torch.take(np.arctanh(m_hat[:, j]), M_set_no_mu).sum() + beta_prior)

In [318]:
m

tensor([[ 0.3322,  0.3995,  0.0378,  ..., -0.6979,  0.2202,  0.4450],
        [ 0.3322,  0.3995,  0.0378,  ..., -0.6979,  0.2202,  0.4450],
        [ 0.3322,  0.3995,  0.0378,  ..., -0.6979,  0.2202,  0.4450],
        ...,
        [ 0.3322,  0.3995,  0.0378,  ..., -0.6979,  0.2202,  0.4450],
        [ 0.3322,  0.3995,  0.0378,  ..., -0.6979,  0.2202,  0.4450],
        [ 0.3322,  0.3995,  0.0378,  ..., -0.6979,  0.2202,  0.4450]])

The next step is to implement the pseudo-posterior, which is calculated after convergence of the messages above:

$$ m_{j} = \tanh \left(  \sum_{ \nu \in {\cal M} (j) } \tanh^{-1} \left(  \hat{m}_{\nu j}  \right)    + \beta(\rho_\xi)  \right)   \; \; ,  $$

In [319]:
m_bayes = []

for j in range(N):
    
    M_set = torch.where(encoding == j)[0]
       
    m_j = torch.tanh(torch.take(np.arctanh(m_hat[:, j]), M_set).sum() + beta_prior)
    
    m_bayes.append(m_j.item())
            
m_bayes

[0.33222657442092896,
 0.3994652032852173,
 0.03776794299483299,
 0.8010807037353516,
 0.5723450779914856,
 -0.6891608834266663,
 0.03752218559384346,
 0.09448682516813278,
 0.6110804080963135,
 0.05468584597110748,
 0.4275072515010834,
 -0.6356916427612305,
 0.5024333000183105,
 0.091519795358181,
 0.7058322429656982,
 0.8075684309005737,
 0.5458797216415405,
 0.8480271697044373,
 0.0021809006575495005,
 -0.24809210002422333,
 0.05085627734661102,
 0.36619940400123596,
 0.4009995460510254,
 -0.1993909478187561,
 0.8865303993225098,
 0.43798211216926575,
 -0.2457575798034668,
 -0.418385773897171,
 -0.3846236765384674,
 -0.46582040190696716,
 0.012754389084875584,
 0.4331659972667694,
 -0.11854452639818192,
 0.5488676428794861,
 0.13262246549129486,
 0.734569251537323,
 0.8222662210464478,
 0.4471621513366699,
 0.19506648182868958,
 0.8252844214439392,
 -0.032206062227487564,
 -0.6202975511550903,
 -0.033446304500103,
 -0.0010305192554369569,
 0.732318103313446,
 0.33630698919296265,
 0

In [320]:
len(m_bayes)

100

In [321]:
np.sign(m_bayes)

array([ 1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1., -1.,  1.,  1.,
       -1., -1., -1., -1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1., -1., -1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,
       -1., -1., -1.,  1.,  1.,  1., -1., -1., -1., -1.,  1.,  1., -1.,
        1.,  1., -1.,  1.,  1.,  1., -1.,  1.,  1., -1.,  1.,  1.,  1.,
        1., -1.,  1.,  1.,  1.,  1.,  1., -1., -1., -1., -1., -1., -1.,
        1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1.])

Gathering all message iterations on functions:

In [322]:
def m_to_mhat(N, M, J_, m, beta, encoding):
    
    m_hat = torch.empty(M, N)
    
    for mu in range(M):
        for j in range(N):
                     
            # Keep only L(mu) which a are different of j
            index_no_j = torch.nonzero(encoding[mu] != j).squeeze()
            L_no_j = encoding[mu][index_no_j]
        
            # Message update        
            m_hat[mu, j] = torch.tanh( beta* J_[mu])*torch.take(m[mu], L_no_j).prod(dim=0)
            
    return m_hat


#####################################################################


def mhat_to_m(N, M, m_hat, beta_prior, encoding):
    
    m = torch.empty(M, N)
    
    for j in range(N):
        
        for mu in range(M):
            
            M_set = torch.where(encoding == j)[0]
        
            M_set_no_mu = M_set[torch.nonzero(M_set != mu).squeeze()]
            
            m[mu, j] = torch.tanh(torch.take(np.arctanh(m_hat[:, j]), M_set_no_mu).sum() + beta_prior)
            
    return m


#####################################################################


def m_bayes(N, m_hat_final, beta_prior, encoding):
    
    m_bayes_ = []

    for j in range(N):
           
        M_set = torch.where(encoding == j)[0]
  
        m_j = torch.tanh(torch.take(np.arctanh(m_hat_final[:, j]), M_set).sum() + beta_prior)
    
        m_bayes_.append(m_j.item())
        
    return m_bayes_

Now we construct a function for the iterative decoding.

In [341]:
def BP_LDPC(N, M, J_, beta, beta_prior, encoding, message, num_it= 100, verbose= 1):
     
    # Initializing
    
    m = torch.ones([M, N])*np.tanh(beta_prior)
    m_hat = torch.ones([M, N])*0.0
    
    for j in range(num_it):
        
        print('-it = %d' % j)

        m_hat = m_to_mhat(N, M, J_, m, beta, encoding)
        #print('m_hat= ', m_hat)
        m = mhat_to_m(N, M, m_hat, beta_prior, encoding)
        #print('m= ', m)
        
        ## Monitoring performace
        if (j % verbose) == 0:
            m_ = m_bayes(N, m_hat, beta_prior, encoding)
            print('overlap= ', torch.dot(message, torch.Tensor(np.sign(m_))).item() / N)
            #print('decoded message_= ', m_)
            print('decoded message= ', np.sign(m_))
        
    m_final = m_bayes(N, m_hat, beta_prior, encoding)
    
    return np.sign(m_final)

In [324]:
opt_dec_Bayes = BP_LDPC(N, M, J_, beta, beta_prior, encoding, message[0], num_it= 10, verbose= 1)

--it = 0
overlap=  -0.12
decoded message=  [ 1. -1. -1.  1.  1. -1. -1. -1.  1. -1.  1. -1.  1. -1.  1.  1.  1.  1.
 -1.  1.  1. -1.  1. -1.  1.  1. -1.  1. -1.  1.  1. -1.  1.  1. -1.  1.
  1. -1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.
  1. -1.  1.  1. -1. -1. -1. -1.  1.  1. -1. -1. -1. -1. -1. -1. -1. -1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.  1. -1. -1. -1. -1.  1.
 -1. -1.  1. -1.  1.  1. -1.  1.  1.  1.]
--it = 1
overlap=  0.06
decoded message=  [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.
  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.  1.
  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
 -1.  1.  1.  1.  1.  1.  1. -1.  1.  1.]
--it = 2
overlap=  0.04
decoded message=  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1

In [325]:
opt_dec_Bayes

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

Ok. It is still slow (we will worrie about this latter), but it appears to work.

Observe that we arbitrarily consider a certain `num_it`. Naturally, a important question remains: how do we monitor the convergence of BP?

## Loop over different noise parameters to chech consistency

In [342]:
# Message lengh
N = 250

# Codeword lengh
M = 1000

# Non-zero elements per row of the generation matrix
K = 5

# Number of messages
n = 100

# Noisy channel
p = np.array([0.1, 0.2, 0.225, 0.25, 0.275, 0.3, 0.325, 0.35, 0.375, 0.4, 0.45, 0.495])
beta = 0.5*np.log( (1 - p) / p)

# Message prior
p_prior = 0.1
beta_prior = 0.5*np.log( (1 - p_prior) / p_prior)

Generating a message.

In [343]:
random = torch.rand([n, N])

#### -1 with probability p_prior
#### +1 with probability 1 - p_prior

message = 2* (random > p_prior).float() - 1

In [344]:
message

tensor([[-1.,  1.,  1.,  ...,  1.,  1., -1.],
        [ 1.,  1.,  1.,  ...,  1.,  1., -1.],
        [ 1.,  1.,  1.,  ...,  1., -1.,  1.],
        ...,
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.]])

Encoding.

In [345]:
encoding = torch.randint(0, N, [M, K])

In [346]:
encoding

tensor([[ 97,  52, 171,   3, 167],
        [137, 244, 123,  97, 174],
        [210, 162, 131,  61,  78],
        ...,
        [  8, 226, 180,  38, 109],
        [ 38,  65,  88,  79, 169],
        [233,  73, 224, 132, 121]])

In [347]:
# Initializing
J0 = torch.take(message[0], encoding).prod(dim=1)
J0 = J0.unsqueeze(0)

# Loop over all messages
for j in range(1, message.shape[0]):
    
    J0_ = torch.take(message[j], encoding).prod(dim=1)
    J0_ = J0_.unsqueeze(0)
    
    J0 = torch.cat((J0, J0_), dim= 0)

In [348]:
J0

tensor([[-1., -1.,  1.,  ..., -1.,  1.,  1.],
        [ 1.,  1.,  1.,  ..., -1., -1., -1.],
        [ 1.,  1., -1.,  ...,  1.,  1.,  1.],
        ...,
        [ 1., -1.,  1.,  ...,  1.,  1.,  1.],
        [ 1., -1.,  1.,  ...,  1., -1., -1.],
        [ 1.,  1., -1.,  ...,  1., -1., -1.]])

In [349]:
J0.shape

torch.Size([100, 1000])

In [350]:
J0

tensor([[-1., -1.,  1.,  ..., -1.,  1.,  1.],
        [ 1.,  1.,  1.,  ..., -1., -1., -1.],
        [ 1.,  1., -1.,  ...,  1.,  1.,  1.],
        ...,
        [ 1., -1.,  1.,  ...,  1.,  1.,  1.],
        [ 1., -1.,  1.,  ...,  1., -1., -1.],
        [ 1.,  1., -1.,  ...,  1., -1., -1.]])

Loop.

In [351]:
random = torch.rand(J0.shape)

p_list = []
overlap_list = []

for p_ in p:
    
    print('>----- p = ', p_)
    
    p_list.append(p_)
    
    beta_ = 0.5*np.log( (1 - p_) / p_)
    
    ## Flip tensor: -1 with probability p
    flip = 2*(random > p_).float() - 1
    ## J corrupted version: element wise multiplication
    J = torch.mul(J0, flip)
    ##
    overlap_list_p = []

    for k in range(int(J.shape[0] / 20 )):
    
        print('--message %d' % k)
       
        opt_dec_Bayes = BP_LDPC(N, M, J[k], beta_, beta_prior, encoding, message[k], num_it= 8, verbose= 1)
    
        overlap_list_p.append(opt_dec_Bayes)
        
    overlap_list.append(opt_dec_Bayes)

>----- p =  0.1
--message 0
-it = 0
overlap=  0.92
decoded message=  [-1.  1.  1. -1. -1. -1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1. -1.
  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
 -1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1. -1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1. -1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1. -1.  1.  1. -1.  1.
  1.  1.  1.  1.  1.  1. -1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1. -1. -1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1. -1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.  1. -1.  1.  1.
  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1

overlap=  0.896
decoded message=  [ 1.  1.  1.  1.  1.  1. -1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1. -1.  1.  1. -1.  1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.  1.
  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1. 

overlap=  0.936
decoded message=  [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1. -1.  1.  1.  1.  1. -1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1. -1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.
  1.  1. -1. -1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1.  1.
  1.  1. -1.  1. 

overlap=  0.936
decoded message=  [ 1.  1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.
  1. -1.  1.  1. -1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
 -1.  1. -1. -1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1. -1.
  1.  1.  1. -1.  1.  1. -1.  1.  1.  1.  1. -1.  1.  1.  1.  1. -1.  1.
 -1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1. -1.  1. -1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.
  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1. -1.
  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.  1.
  1.  1. -1.  1. 

overlap=  0.912
decoded message=  [ 1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
 -1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.
  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1. -1.  1.  1.  1.  1.
  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1.  1.
  1.  1. -1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1. -1.
  1.  1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1. -1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1. -1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1. 

...

overlap=  0.792
decoded message=  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
-it = 2
overlap=  0.792
decoded message=  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 

overlap=  0.76
decoded message=  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
-it = 5
overlap=  0.76
decoded message=  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1.

overlap=  0.792
decoded message=  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
--message 2
-it = 0
overlap=  0.808
decoded message=  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 

overlap=  0.808
decoded message=  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
-it = 3
overlap=  0.808
decoded message=  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 

overlap=  0.792
decoded message=  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
-it = 6
overlap=  0.792
decoded message=  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 

In [352]:
p_list

[0.1, 0.2, 0.225, 0.25, 0.275, 0.3, 0.325, 0.35, 0.375, 0.4, 0.45, 0.495]

In [353]:
overlap_list

[[array([-1.,  1.,  1., -1., -1., -1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.,  1., -1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,
          1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1., -1.,
          1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1., -1.,  1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1., -1.,  1., -1.,  1.,

In [355]:
rho_list = []

In [357]:
# p = 0.1
rho = [1.0, 1.0, 1.0, 1.0, 1.0]
rho_list.append(rho)

In [359]:
# p = 0.2
rho = [1.0, 1.0, 1.0, 0.992, 0.992]
rho_list.append(rho)

In [360]:
# p = 0.225
rho = [1.0, 0.968, 1.0, 0.984, 0.984]
rho_list.append(rho)

In [361]:
# p = 0.25
rho = [0.992, 0.912, 0.992, 0.984, 0.984]
rho_list.append(rho)

In [362]:
# p = 0.275
rho = [0.992, 0.88, 0.984, 0.976, 0.92]
rho_list.append(rho)

In [363]:
# p = 0.3
rho = [0.928, 0.784, 0.96, 0.912, 0.848]
rho_list.append(rho)

In [364]:
# p = 0.325
rho = [0.776, 0.832, 0.904, 0.864, 0.856]
rho_list.append(rho)

In [365]:
# p = 0.35
rho = [0.768, 0.792, 0.864, 0.808, 0.8]
rho_list.append(rho)

In [366]:
# p = 0.375
rho = [0.752, 0.792, 0.84, 0.8, 0.792]
rho_list.append(rho)

In [367]:
# p = 0.4
rho = [0.792, 0.792, 0.816, 0.808, 0.784]
rho_list.append(rho)

In [368]:
# p = 0.45
rho = [0.76, 0.792, 0.808, 0.808, 0.792]
rho_list.append(rho)

In [369]:
# p = 0.495
rho = [0.76, 0.792, 0.808, 0.808, 0.792]
rho_list.append(rho)

In [370]:
rho_list

[[1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 0.992, 0.992],
 [1.0, 0.968, 1.0, 0.984, 0.984],
 [0.992, 0.912, 0.992, 0.984, 0.984],
 [0.992, 0.88, 0.984, 0.976, 0.92],
 [0.928, 0.784, 0.96, 0.912, 0.848],
 [0.776, 0.832, 0.904, 0.864, 0.856],
 [0.768, 0.792, 0.864, 0.808, 0.8],
 [0.752, 0.792, 0.84, 0.8, 0.792],
 [0.792, 0.792, 0.816, 0.808, 0.784],
 [0.76, 0.792, 0.808, 0.808, 0.792],
 [0.76, 0.792, 0.808, 0.808, 0.792]]

In [382]:
rho_list_av = [np.mean(rho_list[j]) for j in range(len(rho_list))]

In [383]:
rho_list_av

[1.0,
 0.9968,
 0.9872,
 0.9728,
 0.9503999999999999,
 0.8864000000000001,
 0.8464,
 0.8064,
 0.7952,
 0.7984,
 0.792,
 0.792]

In [387]:
rho_list_av_f = ['{:.6f}'.format(rho_list_av[j]) for j in range(len(rho_list))]
p_list_f = ['{:.3f}'.format(p_list[j]) for j in range(len(p_list))]

In [388]:
with open('p_versus_overlap_BP_N_250_M_1000_p_prior_0100_messages_5_it_8.txt', 'w') as f:
    for j in range(len(rho_list_av_f)):
        f.write(p_list_f[j] + '  ' + rho_list_av_f[j] + '\n')